In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import nd2reader
import numpy as np
from cytoolz import compose
import skimage
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from paulssonlab.projects.molecule_counting.segmentation import segment, invert
from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels
import cv2

mpl.rcParams["figure.figsize"] = (14, 14)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch2/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[2]
print(file)
nd2 = nd2reader.ND2Reader(file)
print("Channels: ", nd2.metadata["channels"])
# print('Fields of view: ', nd2.metadata['fields_of_view'])
# print(nd2.metadata)
print(nd2.sizes)

In [ ]:
fov_index = 1
phase_image = nd2.get_frame_2D(v=0, c=0, z=fov_index)
fluor_image = nd2.get_frame_2D(v=0, c=1, z=fov_index)
plt.imshow(phase_image)

In [ ]:
# Crop image
xmin, xmax = 0, 290
ymin, ymax = 0, 172
phase_img = phase_image[ymin:ymax, xmin:xmax]
fluor_img = fluor_image[ymin:ymax, xmin:xmax]
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img, cmap="gray")
ax[1].imshow(fluor_img, cmap="seismic")
# ax[1].imshow(seg_mask, cmap = 'gray_r')

# Segment cells
# seg is a numpy array where 0 = no cell, and number = which cell that point is part of.
seg = segment_phase(phase_img)
seg_mask = np.ma.masked_where(seg != 0, seg)
ax[2].imshow(permute_labels(seg))
# ax[2].imshow(seg)
ax[2].imshow(seg_mask, cmap="gray_r")

In [ ]:
def get_cells(seg):
    """takes in segmented image, returns a list of coordinates included in each cell."""
    number_of_cells = np.amax(seg)
    cell_labels = np.linspace(1, number_of_cells, number_of_cells)
    cells = []
    for cell in cell_labels:
        cell = np.where(seg == cell)
        cells.append(cell)
    return cells


cells = get_cells(seg)
print("Number of cells in fov: ", len(cells))

In [ ]:
def get_cell_boxes(seg):
    """takes in segmented image, returns coordinates of corners of each box with a cell."""
    cells = get_cells(seg)
    cell_boxes = []
    for cell in cells:
        cell_boxes.append(
            [np.amin(cell[0]), np.amax(cell[0]), np.amin(cell[1]), np.amax(cell[1])]
        )
    return cell_boxes


cell_label = 13
cell = cells[cell_label - 1]
boxes = get_cell_boxes(seg)
box = boxes[cell_label - 1]

In [ ]:
# Get images (phase and fluor, and mask) of the cell.
phase_img_cell = phase_img[box[0] : box[1], box[2] : box[3]]
fluor_img_cell = fluor_img[box[0] : box[1], box[2] : box[3]]
seg_box = seg[box[0] : box[1], box[2] : box[3]]
grey_fluor_img_cell = rgb2gray(fluor_img_cell)

grey_fluor_img_cell_masked = np.ma.masked_where(
    seg_box != cell_label, grey_fluor_img_cell
)
phase_img_cell_masked = np.ma.masked_where(seg_box != cell_label, phase_img_cell)
fluor_img_cell_masked = np.ma.masked_where(seg_box != cell_label, fluor_img_cell)
fig, axes = plt.subplots(1, 6, figsize=(25, 10), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img_cell)
ax[0].set_title("Phase image")
ax[1].imshow(phase_img_cell_masked)
ax[1].set_title("Masked phase image")
ax[2].imshow(fluor_img_cell)
ax[2].set_title("Fluorescence image")
ax[3].imshow(fluor_img_cell_masked)
ax[3].set_title("Masked fluorescence image")
ax[4].imshow(grey_fluor_img_cell)
ax[4].set_title("Grayscale fluorescence image")
ax[5].imshow(grey_fluor_img_cell_masked)
ax[5].set_title("Masked grayscale fluorescence image")

In [ ]:
def mean_background_int(seg_img):
    """Takes in a segmented image and returns the mean intensity of the background."""
    background = np.ma.masked_where(seg_img.mask == False, seg_img.data)
    return np.mean(background)


print(mean_background_int(grey_fluor_img_cell_masked))

In [ ]:
def std_background_int(seg_img):
    """Takes in a segmented image and returns the standard deviation of the background
    intensity, so we can get an idea of the noise level."""
    background = np.ma.masked_where(seg_img.mask == False, seg_img.data)
    return np.std(background)


print(std_background_int(grey_fluor_img_cell_masked))

In [ ]:
mean_bg_int = mean_background_int(grey_fluor_img_cell_masked)
grey_fluor_img_cell_masked_bc = grey_fluor_img_cell_masked - mean_bg_int
plt.figure(figsize=(5, 5))
plt.imshow(grey_fluor_img_cell_masked_bc)

In [ ]:
mean_bg_int = mean_background_int(grey_fluor_img_cell_masked)
grey_fluor_img_cell_masked_bc = grey_fluor_img_cell_masked - mean_bg_int
plt.figure(figsize=(5, 5))
plt.imshow(grey_fluor_img_cell_masked_bc)

In [ ]:
def get_blobs_log(img):
    blobs_log = blob_log(img, max_sigma=10, threshold=1000)
    blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
    return blobs_log


def get_blobs_dog(img):
    blobs_dog = blob_dog(img, max_sigma=10, threshold=1000)
    blobs_dog[:, 2] = blobs_dog[:, 2] * np.sqrt(2)
    return blobs_dog


def get_blobs_doh(img):
    return blob_doh(grey_fluor_img_cell_masked, max_sigma=10, threshold=0.01)


blobs_list = [
    get_blobs_log(grey_fluor_img_cell_masked_bc),
    get_blobs_dog(grey_fluor_img_cell_masked_bc),
    get_blobs_doh(grey_fluor_img_cell_masked_bc),
]
print(blobs_list)
colors = ["cyan", "lime", "red"]
titles = ["Laplacian of Gaussian", "Difference of Gaussian", "Determinant of Hessian"]
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(20, 40), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(grey_fluor_img_cell_masked, cmap="gray")
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
        ax[idx].plot(x, y, "o", color=color)
#     ax[idx].set_axis_off()

In [ ]:
def get_blob_coordinates(blob_list):
    """Takes in lists of lists of blob coordinates and radii, as outputted by get_blobs_... and returns
    a list of lists of coordinates that are within those circles."""
    coordinates = []
    for blob in blob_list:
        y, x, r = blob
        ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
        rel_points = np.where(
            (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
        )
        points = [rel_points[0] + x - np.ceil(r), rel_points[1] + y - np.ceil(r)]
        coordinates.append(points)
    return coordinates


test = get_blob_coordinates(blobs_list[0])
plt.figure(figsize=(5, 5))
plt.imshow(grey_fluor_img_cell_masked_bc, cmap="gray")
for i in range(len(test)):
    plt.plot(*test[i], "r.")

In [ ]:
def total_fluor_ints_blobs(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the total fluorescence intensities
    within those blobs."""
    ints = []
    for i in range(len(blob_coordinates)):
        blob = blob_coordinates[i]
        ints.append(np.sum(img[list(map(int, blob[1])), list(map(int, blob[0]))]))
    return ints


print(
    total_fluor_ints_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def total_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the total fluorescence intensity
    outside the blobs. I.e. total fluorescence of the rest of the cell."""
    total_fluor = np.sum(img)
    blob_ints = total_fluor_ints_blobs(img, blob_coordinates)
    total_fluor -= np.sum(blob_ints)
    return total_fluor


print(
    total_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def fraction_int_in_blobs(img, blob_coordinates):
    total_fluor = np.sum(img)
    total_blob_ints = np.sum(total_fluor_ints_blobs(img, blob_coordinates))
    return total_blob_ints / total_fluor


print(
    fraction_int_in_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_ints_blobs(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensities
    within those blobs."""
    means = []
    for i in range(len(blob_coordinates)):
        blob = blob_coordinates[i]
        means.append(np.mean(img[list(map(int, blob[1])), list(map(int, blob[0]))]))
    return means


print(
    mean_fluor_ints_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensity
    outside the blobs. I.e. mean fluorescence of the rest of the cell."""
    bg_total_fluor = total_fluor_int_background(img, blob_coordinates)
    bg_points = np.ma.count(img)
    for blob in blob_coordinates:
        bg_points -= len(blob[0])
    return bg_total_fluor / bg_points


print(
    mean_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def total_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the total fluorescence intensity
    outside the blobs. I.e. total fluorescence of the rest of the cell."""
    total_fluor = np.sum(img)
    blob_ints = total_fluor_ints_blobs(img, blob_coordinates)
    total_fluor -= np.sum(blob_ints)
    return total_fluor


print(
    total_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def fraction_int_in_blobs(img, blob_coordinates):
    total_fluor = np.sum(img)
    total_blob_ints = np.sum(total_fluor_ints_blobs(img, blob_coordinates))
    return total_blob_ints / total_fluor


print(
    fraction_int_in_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_ints_blobs(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensities
    within those blobs."""
    means = []
    for i in range(len(blob_coordinates)):
        blob = blob_coordinates[i]
        means.append(np.mean(img[list(map(int, blob[1])), list(map(int, blob[0]))]))
    return means


print(
    mean_fluor_ints_blobs(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)

In [ ]:
def mean_fluor_int_background(img, blob_coordinates):
    """Takes in the masked image and a list of coordinates for each blob and returns the mean fluorescence intensity
    outside the blobs. I.e. mean fluorescence of the rest of the cell."""
    bg_total_fluor = total_fluor_int_background(img, blob_coordinates)
    bg_points = np.ma.count(img)
    for blob in blob_coordinates:
        bg_points -= len(blob[0])
    return bg_total_fluor / bg_points


print(
    mean_fluor_int_background(
        grey_fluor_img_cell_masked_bc, get_blob_coordinates(blobs_list[0])
    )
)